# tutorials

> How to use the tool

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from slicer_test.core import *
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import PIL,os,mimetypes, glob, skimage
from PIL import Image
from natsort import natsorted

## Parameters

> Important: Please define all parameters below

> Important: The function ImageSlicer.show_slices() will help to visualize the scaling

> Important: For the parameters of the functions see Core

> Note: The scaling of different datasets can be adjusted -> Improves the stability of the training significantly